In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

Below I have code to collect all the parquet files in a list and than concataning the files together and printing them in a wrong way. But because I didn't want to keep doing it. It is commented out. I than have the file being read into them.Than I wanted to check shape and columns.

In [2]:
cleaneddata = pd.read_parquet('cleaneddata.parquet')
cleaneddata.shape

(37564080, 23)

## All below are just for analysing the data. 
i broke them down into months, hours, and season, holidays. 

Ok firstly I want the month and I just want to order them because it wasnt ordered in mine. And change it to category this is just for analysis though.

In [3]:
# Extract the month from the datetime columns
cleaneddata['pickup_month'] = cleaneddata['tpep_pickup_datetime'].dt.month_name()
cleaneddata['dropoff_month'] = cleaneddata['tpep_dropoff_datetime'].dt.month_name()

# Define the desired order of months
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Convert the 'pickup_month' and 'dropoff_month' columns to categorical type with the specified order
cleaneddata['pickup_month'] = pd.Categorical(cleaneddata['pickup_month'], categories=month_order, ordered=True)
cleaneddata['dropoff_month'] = pd.Categorical(cleaneddata['dropoff_month'], categories=month_order, ordered=True)

# Sort the DataFrame by the 'pickup_month' column
df = cleaneddata.sort_values('pickup_month')

This one is to seperate the time and date into two different columns. Just for easier analysis latewr on

I have this test to see if the trip was in one day or a few days like. I wanted to see if it was plausible .As in maybe the trip duration 

In [4]:
def round_to_nearest_hour(time):
    rounded_time = time.replace(minute=0, second=0)
    return rounded_time

df['dropoff_time'] = df['dropoff_time'].apply(round_to_nearest_hour)
df['pickup_time'] = df['pickup_time'].apply(round_to_nearest_hour)

In [5]:
# Identify holidays
cal = calendar()
holidays = cal.holidays(start='2022-01-01', end='2022-12-31')
# Convert 'pickup_date' to datetime-like object
df['pickup_date'] = pd.to_datetime(df['pickup_date'])

# Check if the pickup date is a holiday and assign day types
df['holiday'] = df['pickup_date'].isin(holidays)
df['week'] = df['pickup_date'].dt.dayofweek
df.loc[df['week'] >= 5, 'day_type'] = "weekend"
df.loc[df['week'] < 5, 'day_type'] = "workday"
df.loc[df['holiday'] == True, 'day_type'] = "holiday"

In [6]:
print(df.head())

          VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
5464533          2  2009-01-01 09:33:43   2009-01-01 10:22:43   
10873564         1  2022-01-13 08:19:26   2022-01-13 08:27:05   
10873563         2  2022-01-13 08:39:42   2022-01-13 08:46:52   
10873562         2  2022-01-13 08:53:15   2022-01-13 08:58:41   
10873561         2  2022-01-13 08:25:16   2022-01-13 08:32:56   

          passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
5464533               5.0          10.70         1.0                  N   
10873564              1.0           1.20         1.0                  N   
10873563              1.0           1.22         1.0                  N   
10873562              1.0           0.74         1.0                  N   
10873561              1.0           1.39         1.0                  N   

          PULocationID  DOLocationID  payment_type  ...  airport_fee  \
5464533            138           162             1  ...          0.0   
10873564      

In [7]:
# # Drop the 'holiday' and 'week' columns
# df = df.drop(['holiday', 'week'], axis=1)

# Define a function to assign time slots
def time_slots(x):
    if x.hour in range(6, 12):
        return 'Morning'
    elif x.hour in range(12, 17):
        return 'Afternoon'
    elif x.hour in range(17, 22):
        return 'Evening'
    else:
        return 'Late Night'

# Apply the time_slots function to 'tpep_pickup_datetime'
df['dropoff_timeslots'] = df['dropoff_time'].apply(time_slots)
df['pickup_timeslots'] = df['pickup_time'].apply(time_slots)

# Print the updated DataFrame
# print(df.head())

Ok below is seasons just to make sure I have all the seasons here so that I can analysis the data.

In [8]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
pickup_date              datetime64[ns]
pickup_time                      object
dropoff_date                     object
dropoff_time                     object
pickup_month                   category
dropoff_month                  category


In [9]:
# dataset_year = pd.to_datetime(df['tpep_pickup_datetime']).dt.year.unique()[0]

# winter_start = pd.to_datetime('2022-01-01').date()
# winter_end = pd.to_datetime('2022-02-28').date()
# spring_start = pd.to_datetime('2022-03-01').date()
# spring_end = pd.to_datetime('2022-05-31').date()
# summer_start = pd.to_datetime('2022-06-01').date()
# summer_end = pd.to_datetime('2022-08-31').date()
# autumn_start = pd.to_datetime('2022-09-01').date()
# autumn_end = pd.to_datetime('2022-11-30').date()

# # Set the 'season' value based on the date range
# df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
# df['season'] = pd.NA
# df.loc[(df['tpep_pickup_datetime'] >= winter_start) & (df['tpep_pickup_datetime'] <= winter_end), 'season'] = 'winter'
# df.loc[(df['tpep_pickup_datetime'] >= spring_start) & (df['tpep_pickup_datetime'] <= spring_end), 'season'] = 'spring'
# df.loc[(df['tpep_pickup_datetime'] >= summer_start) & (df['tpep_pickup_datetime'] <= summer_end), 'season'] = 'summer'
# df.loc[(df['tpep_pickup_datetime'] >= autumn_start) & (df['tpep_pickup_datetime'] <= autumn_end), 'season'] = 'autumn'

# # Filter the DataFrame for each season
# winter_data = df[df['season'] == 'winter']
# spring_data = df[df['season'] == 'spring']
# summer_data = df[df['season'] == 'summer']
# autumn_data = df[df['season'] == 'autumn']

# # Count the number of NaN values in the 'season' column
# nan_counts = df['season'].isna().sum()

# # Print the results
# print("Number of NaN values in 'season' column:", nan_counts)

In [10]:
# Extract the month from the pickup datetime column
df['pickup_month'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.month_name()

# Set the 'season' value based on the month
df['season'] = pd.NA
df.loc[df['pickup_month'].isin(['December', 'January', 'February']), 'season'] = 'winter'
df.loc[df['pickup_month'].isin(['March', 'April', 'May']), 'season'] = 'spring'
df.loc[df['pickup_month'].isin(['June', 'July', 'August']), 'season'] = 'summer'
df.loc[df['pickup_month'].isin(['September', 'October', 'November']), 'season'] = 'autumn'

# Filter the DataFrame for each season
winter_data = df[df['season'] == 'winter']
spring_data = df[df['season'] == 'spring']
summer_data = df[df['season'] == 'summer']
autumn_data = df[df['season'] == 'autumn']

# Count the number of NaN values in the 'season' column
nan_counts = df['season'].isna().sum()

# Print the results
print("Number of NaN values in 'season' column:", nan_counts)

Number of NaN values in 'season' column: 0


In [11]:
# Get the unique values for the "passenger" column
passenger_values = df['passenger_count'].unique()

# Print the unique values
print("Unique passenger values:")
print(passenger_values)

# Count the occurrences of each unique value in the "passenger" column
passenger_counts = df['passenger_count'].value_counts()

# Print the counts
print("Value counts for passenger:")
print(passenger_counts)

Unique passenger values:
[5. 1. 0. 4. 3. 6. 2. 8. 9. 7.]
Value counts for passenger:
passenger_count
1.0    27709407
2.0     5767000
3.0     1514038
0.0      746241
4.0      690121
5.0      680918
6.0      456204
7.0          79
8.0          55
9.0          17
Name: count, dtype: int64


In [12]:
passenger_counts = df['passenger_count'].value_counts()

# Calculate the percentage for each value
passenger_percentage = passenger_counts / len(df) * 100

# Print the percentages
print("Percentage for each passenger count:")
print(passenger_percentage)

Percentage for each passenger count:
passenger_count
1.0    73.765701
2.0    15.352432
3.0     4.030547
0.0     1.986581
4.0     1.837183
5.0     1.812684
6.0     1.214469
7.0     0.000210
8.0     0.000146
9.0     0.000045
Name: count, dtype: float64


In [13]:
print(df.dtypes) 

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
pickup_date              datetime64[ns]
pickup_time                      object
dropoff_date                     object
dropoff_time                     object
pickup_month                     object
dropoff_month                  category


In [14]:
# Assuming 'combined_df' is your DataFrame and 'column1', 'column2', 'column3' are the columns you want to drop
combination = df.drop(['trip_distance','VendorID', 'tolls_amount', 'extra','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','airport_fee','store_and_fwd_flag','tip_amount','mta_tax','payment_type','fare_amount','RatecodeID'], axis=1)

In [15]:
print(combination)

         tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
5464533   2009-01-01 09:33:43   2009-01-01 10:22:43              5.0   
10873564  2022-01-13 08:19:26   2022-01-13 08:27:05              1.0   
10873563  2022-01-13 08:39:42   2022-01-13 08:46:52              1.0   
10873562  2022-01-13 08:53:15   2022-01-13 08:58:41              1.0   
10873561  2022-01-13 08:25:16   2022-01-13 08:32:56              1.0   
...                       ...                   ...              ...   
25186517  2022-12-27 10:58:39   2022-12-27 11:03:38              3.0   
25186516  2022-12-27 10:29:43   2022-12-27 10:38:31              1.0   
25186515  2022-12-27 10:19:25   2022-12-27 10:25:40              2.0   
25186537  2022-12-27 10:44:25   2022-12-27 10:47:30              1.0   
23004455  2022-12-06 17:37:16   2022-12-06 19:01:46              2.0   

          PULocationID  DOLocationID pickup_date pickup_time dropoff_date  \
5464533            138           162  2009-01-01    09:00:

In [16]:
nan_counts = combination.isna().sum()
print("NaN counts for each column:")
print(nan_counts)

NaN counts for each column:
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
PULocationID             0
DOLocationID             0
pickup_date              0
pickup_time              0
dropoff_date             0
dropoff_time             0
pickup_month             0
dropoff_month            0
holiday                  0
week                     0
day_type                 0
dropoff_timeslots        0
pickup_timeslots         0
season                   0
dtype: int64


In [17]:
# Drop rows with any missing values
combinations = combination.dropna()

In [18]:
print(combinations.columns)
filtered_table = pa.Table.from_pandas(combinations)
pq.write_table(filtered_table, 'dataforanalysing.parquet')

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'PULocationID', 'DOLocationID', 'pickup_date', 'pickup_time',
       'dropoff_date', 'dropoff_time', 'pickup_month', 'dropoff_month',
       'holiday', 'week', 'day_type', 'dropoff_timeslots', 'pickup_timeslots',
       'season'],
      dtype='object')
